In [2]:
import pandas as pd
import torch
import torch.nn as nn
from torchsummary import summary
from sklearn.model_selection import train_test_split 
from torch.utils.data import DataLoader, TensorDataset
import matplotlib.pyplot as plt

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
df = pd.read_csv('../data/ecom-user-churn-data.csv')

df = df.drop(['visitorid', 'int_cat15_n'], axis = 1)
X = df.drop(['target_class'], axis=1)
y = df['target_class']

In [5]:
BATCH_SIZE = 50

X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.20, 
                                                    random_state=1010, 
                                                    stratify=y)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train,
                                                    test_size=0.20, 
                                                    random_state=1010, 
                                                    stratify=y_train)

# train data
X_train = torch.tensor(X_train.values)
y_train = torch.tensor(y_train.values)
dataset_train = TensorDataset(X_train, y_train)
dataloader_train = DataLoader(dataset_train, batch_size=BATCH_SIZE, shuffle=True)

# validation data
X_val = torch.tensor(X_val.values)
y_val = torch.tensor(y_val.values)
dataset_val = TensorDataset(X_val, y_val)
dataloader_val = DataLoader(dataset_val, batch_size=BATCH_SIZE, shuffle=True)


X_test = torch.tensor(X_test.values)
y_test = torch.tensor(y_test.values)
dataset_test = TensorDataset(X_test, y_test)
dataloader_test = DataLoader(dataset_test, batch_size=BATCH_SIZE, shuffle=True)

In [87]:
class SimpleNN(nn.Module):
    def __init__(self, input_size, hidden_size_1, hidden_size_2, output_size):
        super().__init__()
        
        self.main = torch.nn.Sequential(
            nn.Linear(input_size, hidden_size_1),
            nn.ReLU(), 
            nn.Dropout(0.2),
            nn.Linear(hidden_size_1, hidden_size_2),
            nn.ReLU(), 
            nn.Linear(hidden_size_2, output_size),
            nn.Dropout(0.2)
        )

    def forward(self, x):
        x = self.main(x)
        return x

In [88]:
model = SimpleNN(1, 20, 10, 2)
# summary(model, (1,))
# model.state_dict()

In [91]:
def trainer(model, criterion, optimizer, dataloader_train, dataloader_val, epochs=5, patience=5, verbose=True):
    """Simple training wrapper for PyTorch network."""
    
    per_epoch_train_loss = []
    per_epoch_val_loss = []
    per_epoch_train_accuracy = []
    per_epoch_val_accuracy = []
    consec_increases = 0
    
    for epoch in range(epochs):
        batch_train_loss = 0
        batch_train_acc = 0
        
        # training
        for X_train, y_train in dataloader_train:
            X_train = X_train.float()
            y_train = y_train.long()
            
            optimizer.zero_grad()

            y_hat = model(X_train)
            y_hat_labels = torch.argmax(y_hat, dim=1)

            loss = criterion(y_hat, y_train)
            loss.backward()         
            optimizer.step()
            
            batch_train_loss += loss.item()
            batch_train_acc += (y_hat_labels == y_train).type(torch.float32).mean().item()
        
        # per batch loss & accuracy
        avg_batch_train_loss = batch_train_loss / len(dataloader_train)
        per_epoch_train_loss.append(avg_batch_train_loss)
        
        avg_batch_train_acc = batch_train_acc / len(dataloader_train)
        per_epoch_train_accuracy.append(avg_batch_train_acc)
        
        
        # validation
        model.eval()
        with torch.no_grad():
            batch_val_loss = 0
            batch_val_acc = 0

            for X_val, y_val in dataloader_val:
                X_val = X_val.float()
                y_val = y_val.long()

                y_hat = model(X_val)
                y_hat_labels = torch.argmax(y_hat, dim=1)
                loss = criterion(y_hat, y_val)
                
                batch_val_loss += loss.item()
                batch_val_acc += (y_hat_labels == y_val).type(torch.float32).mean().item()
                        
        # per batch loss
        avg_batch_val_loss = batch_val_loss / len(dataloader_val)
        per_epoch_val_loss.append(avg_batch_val_loss)
                
        avg_batch_val_acc = batch_train_acc / len(dataloader_val)
        per_epoch_val_accuracy.append(avg_batch_val_acc)
        
        model.train()
        
        if verbose: print(f"epoch: {epoch + 1}, train loss: {avg_batch_train_loss:.3f}, val loss: {avg_batch_val_loss:.3f}, train acc: {avg_batch_train_acc:.3f}, val acc: {avg_batch_val_acc:.3f}")
        
        # early stopping
        if epoch > 0 and per_epoch_val_loss[-1] > per_epoch_val_loss[-2]:
            consec_increases =+ 1
        
        else: 
            consec_increases = 0
        
        if consec_increases == patience:
            print(f'Stopped early at epoch {epoch + 1} because val loss increased for {consec_increases} consecutive epochs')
    
    results = {"train_loss": per_epoch_train_loss,
               "valid_loss": per_epoch_val_loss,
               "train_accuracy": per_epoch_train_accuracy,
               "valid_accuracy": per_epoch_val_accuracy}
    
    return results

# set hyperparameters
input_size = 46 # 1 x 46
hidden_size_1 = 10
hidden_size_2 = 5
num_classes = 2
num_epochs = 10
LEARNING_RATE = 0.02

model = SimpleNN(input_size, hidden_size_1, hidden_size_2, num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
trainer(model, criterion, optimizer, dataloader_train, dataloader_val, epochs=num_epochs, verbose=True)


epoch: 1, train loss: 1.972, val loss: 0.327, train acc: 0.851, val acc: 3.405
epoch: 2, train loss: 0.363, val loss: 0.325, train acc: 0.855, val acc: 3.419
epoch: 3, train loss: 0.361, val loss: 0.326, train acc: 0.856, val acc: 3.425
epoch: 4, train loss: 0.360, val loss: 0.335, train acc: 0.855, val acc: 3.420
epoch: 5, train loss: 0.360, val loss: 0.326, train acc: 0.855, val acc: 3.419
epoch: 6, train loss: 0.362, val loss: 0.332, train acc: 0.856, val acc: 3.425
epoch: 7, train loss: 0.363, val loss: 0.331, train acc: 0.857, val acc: 3.426
epoch: 8, train loss: 0.359, val loss: 0.324, train acc: 0.855, val acc: 3.421
epoch: 9, train loss: 0.363, val loss: 0.324, train acc: 0.854, val acc: 3.415
epoch: 10, train loss: 0.364, val loss: 0.328, train acc: 0.853, val acc: 3.413


{'train_loss': [1.9724323920009634,
  0.36336777269651616,
  0.361382598198856,
  0.35989747278958184,
  0.3600430758080528,
  0.3616544528852535,
  0.36294015051349054,
  0.35947065861730637,
  0.3630663968999929,
  0.36424723732037634],
 'valid_loss': [0.3270792544265337,
  0.3251202248885662,
  0.3256573750154127,
  0.3348731731500807,
  0.32600902497202533,
  0.33204614672856997,
  0.33095874363863015,
  0.32365559503624713,
  0.3242533565793611,
  0.3277713791290416],
 'train_accuracy': [0.8512058565510979,
  0.8547068574949156,
  0.8562358379175391,
  0.8550666174556636,
  0.854865084531941,
  0.8561642195416402,
  0.8565539591297319,
  0.8551299086288561,
  0.8537491621284545,
  0.8533460976956766],
 'valid_accuracy': [3.4048234262043917,
  3.4188274299796624,
  3.4249433516701564,
  3.4202664698226544,
  3.419460338127764,
  3.424656878166561,
  3.4262158365189275,
  3.4205196345154243,
  3.414996648513818,
  3.4133843907827064]}